# 04 · Baseline tabular models
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ORG/Fallrisk-gait/blob/main/datasets/fallrisk/notebooks/04_baseline_tabular.ipynb)

Train pure-Python baselines on the synthetic table: logistic regression and gradient boosting stumps for both the binary high-risk label and the 3-class policy. Report AUROC, macro-F1, calibration curves, and demographic slice metrics (sex and age bands).

In [ ]:
from pathlib import Path
import csv
import math
import random
from collections import Counter, defaultdict

def locate_repo_root(max_depth: int = 6) -> Path:
    here = Path.cwd()
    for _ in range(max_depth):
        if (here / 'datasets').exists() and (here / 'data').exists():
            return here
        if here.parent == here:
            break
        here = here.parent
    return Path.cwd()

ROOT = locate_repo_root()
data_path = ROOT / 'datasets' / 'fallrisk' / 'fallrisk_tabular_v1.csv'
with data_path.open() as f:
    rows = list(csv.DictReader(f))
print(f'Loaded {len(rows)} synthetic rows')
random.seed(8)
random.shuffle(rows)
rows = rows[:15000]
split = int(0.8 * len(rows))
train_rows = rows[:split]
test_rows = rows[split:]
FEATURES = ['age_years','bmi','systolic_bp','gait_speed_m_s','postural_sway_cm','medication_count',
             'chronic_conditions','past_falls_6mo','dual_task_cost_percent','fear_of_falling_score',
             'muscle_strength_score','reaction_time_ms']
CLASSES = ['low','moderate','high']


In [ ]:
def to_float(row, key):
    return float(row[key])

class StandardScaler:
    def __init__(self, features):
        self.features = features
        self.stats = {feature: (0.0, 1.0) for feature in features}

    def fit(self, data):
        for feat in self.features:
            vals = [to_float(r, feat) for r in data]
            mean_val = sum(vals) / len(vals)
            var = sum((v - mean_val) ** 2 for v in vals) / len(vals)
            std = math.sqrt(var) if var > 0 else 1.0
            self.stats[feat] = (mean_val, std)

    def transform(self, row):
        return [(to_float(row, feat) - self.stats[feat][0]) / (self.stats[feat][1] or 1.0) for feat in self.features]

def logistic(x):
    return 1.0 / (1.0 + math.exp(-x))

class BinaryLogisticRegression:
    def __init__(self, lr=0.12, epochs=120):
        self.lr = lr
        self.epochs = epochs
        self.weights = None

    def fit(self, X, y):
        self.weights = [0.0] * (len(X[0]) + 1)
        n = len(X)
        for epoch in range(self.epochs):
            grad0 = 0.0
            grad = [0.0] * len(X[0])
            for xi, yi in zip(X, y):
                z = self.weights[0] + sum(w * v for w, v in zip(self.weights[1:], xi))
                p = logistic(z)
                diff = p - yi
                grad0 += diff
                for j in range(len(xi)):
                    grad[j] += diff * xi[j]
            step = self.lr / (1 + 0.01 * epoch)
            self.weights[0] -= step * grad0 / n
            for j in range(len(grad)):
                self.weights[j + 1] -= step * grad[j] / n

    def predict_proba(self, X):
        probs = []
        for xi in X:
            z = self.weights[0] + sum(w * v for w, v in zip(self.weights[1:], xi))
            probs.append(logistic(z))
        return probs

class GradientBoostingStumps:
    def __init__(self, n_estimators=18, learning_rate=0.18):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.trees = []
        self.base_logit = 0.0

    def fit(self, X, y):
        base_rate = sum(y) / len(y)
        base_rate = min(max(base_rate, 1e-6), 1 - 1e-6)
        self.base_logit = math.log(base_rate / (1 - base_rate))
        preds = [self.base_logit] * len(X)
        for _ in range(self.n_estimators):
            probs = [logistic(val) for val in preds]
            residuals = [yt - pr for yt, pr in zip(y, probs)]
            tree = self._fit_stump(X, residuals)
            self.trees.append(tree)
            for i, xi in enumerate(X):
                update = tree['left_value'] if xi[tree['feature']] <= tree['threshold'] else tree['right_value']
                preds[i] += self.learning_rate * update

    def _fit_stump(self, X, residuals):
        best = {'loss': float('inf')}
        total_sum = sum(residuals)
        total_sq = sum(r * r for r in residuals)
        total_count = len(residuals)
        n_features = len(X[0])
        for feat in range(n_features):
            pairs = sorted(((xi[feat], res) for xi, res in zip(X, residuals)), key=lambda x: x[0])
            prefix_sum = 0.0
            prefix_sq = 0.0
            prefix_count = 0
            for i in range(len(pairs) - 1):
                val, res = pairs[i]
                prefix_sum += res
                prefix_sq += res * res
                prefix_count += 1
                if pairs[i + 1][0] == val:
                    continue
                left_count = prefix_count
                right_count = total_count - prefix_count
                if left_count == 0 or right_count == 0:
                    continue
                left_sum = prefix_sum
                right_sum = total_sum - prefix_sum
                left_sq = prefix_sq
                right_sq = total_sq - prefix_sq
                left_mean = left_sum / left_count
                right_mean = right_sum / right_count
                left_loss = left_sq - left_sum * left_sum / left_count
                right_loss = right_sq - right_sum * right_sum / right_count
                loss = left_loss + right_loss
                if loss < best['loss']:
                    threshold = (val + pairs[i + 1][0]) / 2.0
                    best = {
                        'feature': feat,
                        'threshold': threshold,
                        'left_value': left_mean,
                        'right_value': right_mean,
                        'loss': loss
                    }
        if best['loss'] == float('inf'):
            best = {'feature': 0, 'threshold': 0.0, 'left_value': 0.0, 'right_value': 0.0, 'loss': 0.0}
        return best

    def predict_proba(self, X):
        probs = []
        for xi in X:
            logit_val = self.base_logit
            for tree in self.trees:
                update = tree['left_value'] if xi[tree['feature']] <= tree['threshold'] else tree['right_value']
                logit_val += self.learning_rate * update
            probs.append(logistic(logit_val))
        return probs

def auc_score(probs, labels):
    pairs = sorted(zip(probs, labels), key=lambda x: x[0])
    pos = sum(labels)
    neg = len(labels) - pos
    if pos == 0 or neg == 0:
        return None
    rank = 0.0
    cum_pos = 0
    for idx, (score, label) in enumerate(pairs, start=1):
        if label == 1:
            rank += idx
            cum_pos += 1
    return (rank - cum_pos * (cum_pos + 1) / 2) / (pos * neg)

def macro_f1(labels, preds):
    cm = Counter()
    for yt, yp in zip(labels, preds):
        cm[(yt, yp)] += 1
    def f1_for(target):
        tp = cm[(target, target)]
        fp = cm[(1 - target, target)]
        fn = cm[(target, 1 - target)]
        precision = tp / (tp + fp) if tp + fp else 0.0
        recall = tp / (tp + fn) if tp + fn else 0.0
        return 0.0 if precision + recall == 0 else 2 * precision * recall / (precision + recall)
    return 0.5 * (f1_for(0) + f1_for(1))

def calibration_curve(probs, labels, bins=10):
    counts = [0] * bins
    prob_sum = [0.0] * bins
    label_sum = [0.0] * bins
    for p, y in zip(probs, labels):
        idx = min(bins - 1, int(p * bins))
        counts[idx] += 1
        prob_sum[idx] += p
        label_sum[idx] += y
    curve = []
    for i in range(bins):
        if counts[i] == 0:
            curve.append({'bin': i, 'count': 0, 'pred': None, 'actual': None})
        else:
            curve.append({'bin': i, 'count': counts[i], 'pred': prob_sum[i] / counts[i], 'actual': label_sum[i] / counts[i]})
    return curve

def slice_macro_f1(rows, probs, labels, threshold=0.5):
    grouped = defaultdict(list)
    for row, prob, label in zip(rows, probs, labels):
        age = float(row['age_years'])
        if age < 70:
            age_group = '<70'
        elif age < 80:
            age_group = '70-79'
        else:
            age_group = '80+'
        grouped[('sex', row['sex'])].append((prob, label))
        grouped[('age_group', age_group)].append((prob, label))
    metrics = {}
    for key, items in grouped.items():
        preds = [1 if p >= threshold else 0 for p, _ in items]
        true = [y for _, y in items]
        metrics[key] = {
            'count': len(items),
            'macro_f1': round(macro_f1(true, preds), 3)
        }
    return metrics


In [ ]:
scaler = StandardScaler(FEATURES)
scaler.fit(train_rows)
X_train = [scaler.transform(r) for r in train_rows]
y_train = [int(r['label_high_fall_risk']) for r in train_rows]
X_test = [scaler.transform(r) for r in test_rows]
y_test = [int(r['label_high_fall_risk']) for r in test_rows]

logreg = BinaryLogisticRegression(lr=0.15, epochs=140)
logreg.fit(X_train, y_train)
log_probs = logreg.predict_proba(X_test)
log_preds = [1 if p >= 0.5 else 0 for p in log_probs]
log_auc = auc_score(log_probs, y_test)
log_macro_f1 = macro_f1(y_test, log_preds)
print(f'Logistic regression → AUROC: {log_auc:.3f}, macro-F1: {log_macro_f1:.3f}')


In [ ]:
gb = GradientBoostingStumps()
gb.fit(X_train, y_train)
gb_probs = gb.predict_proba(X_test)
gb_preds = [1 if p >= 0.5 else 0 for p in gb_probs]
gb_auc = auc_score(gb_probs, y_test)
gb_macro_f1 = macro_f1(y_test, gb_preds)
print(f'Gradient boosting stumps → AUROC: {gb_auc:.3f}, macro-F1: {gb_macro_f1:.3f}')


In [ ]:
calibration = calibration_curve(log_probs, y_test)
for entry in calibration[:3]:
    print('Calibration bin', entry)


In [ ]:
slices = slice_macro_f1(test_rows, log_probs, y_test)
for key, info in slices.items():
    print('Slice', key, '→', info)


In [ ]:
class OneVsRestLogistic:
    def __init__(self, classes):
        self.classes = classes
        self.models = {}

    def fit(self, X, labels):
        for cls in self.classes:
            binary = [1 if label == cls else 0 for label in labels]
            model = BinaryLogisticRegression(lr=0.12, epochs=120)
            model.fit(X, binary)
            self.models[cls] = model

    def predict(self, X):
        outputs = []
        for xi in X:
            scores = {}
            for cls, model in self.models.items():
                score = model.predict_proba([xi])[0]
                scores[cls] = score
            total = sum(scores.values())
            if total == 0:
                probs = {cls: 1.0 / len(self.classes) for cls in self.classes}
            else:
                probs = {cls: val / total for cls, val in scores.items()}
            outputs.append(probs)
        return outputs

class OneVsRestGB:
    def __init__(self, classes):
        self.classes = classes
        self.models = {}

    def fit(self, X, labels):
        for cls in self.classes:
            binary = [1 if label == cls else 0 for label in labels]
            model = GradientBoostingStumps()
            model.fit(X, binary)
            self.models[cls] = model

    def predict(self, X):
        outputs = []
        for xi in X:
            scores = {}
            for cls, model in self.models.items():
                score = model.predict_proba([xi])[0]
                scores[cls] = score
            total = sum(scores.values())
            if total == 0:
                probs = {cls: 1.0 / len(self.classes) for cls in self.classes}
            else:
                probs = {cls: val / total for cls, val in scores.items()}
            outputs.append(probs)
        return outputs

def macro_f1_multiclass(true_labels, pred_labels):
    scores = []
    for cls in CLASSES:
        tp = sum(1 for yt, yp in zip(true_labels, pred_labels) if yt == cls and yp == cls)
        fp = sum(1 for yt, yp in zip(true_labels, pred_labels) if yt != cls and yp == cls)
        fn = sum(1 for yt, yp in zip(true_labels, pred_labels) if yt == cls and yp != cls)
        precision = tp / (tp + fp) if tp + fp else 0.0
        recall = tp / (tp + fn) if tp + fn else 0.0
        f1 = 0.0 if precision + recall == 0 else 2 * precision * recall / (precision + recall)
        scores.append(f1)
    return sum(scores) / len(scores)

def multiclass_auc(true_labels, prob_dicts):
    aucs = []
    for cls in CLASSES:
        binary_true = [1 if label == cls else 0 for label in true_labels]
        binary_scores = [prob[cls] for prob in prob_dicts]
        auc = auc_score(binary_scores, binary_true)
        if auc is not None:
            aucs.append(auc)
    return sum(aucs) / len(aucs) if aucs else None


In [ ]:
train_labels = [row['label_risk_level'] for row in train_rows]
test_labels = [row['label_risk_level'] for row in test_rows]
ovr_logreg = OneVsRestLogistic(CLASSES)
ovr_logreg.fit(X_train, train_labels)
log_multi_probs = ovr_logreg.predict(X_test)
log_multi_preds = [max(prob.items(), key=lambda x: x[1])[0] for prob in log_multi_probs]
log_multi_f1 = macro_f1_multiclass(test_labels, log_multi_preds)
log_multi_auc = multiclass_auc(test_labels, log_multi_probs)
print(f'Logistic OvR (3-class) → AUROC: {log_multi_auc:.3f}, macro-F1: {log_multi_f1:.3f}')


In [ ]:
ovr_gb = OneVsRestGB(CLASSES)
ovr_gb.fit(X_train, train_labels)
gb_multi_probs = ovr_gb.predict(X_test)
gb_multi_preds = [max(prob.items(), key=lambda x: x[1])[0] for prob in gb_multi_probs]
gb_multi_f1 = macro_f1_multiclass(test_labels, gb_multi_preds)
gb_multi_auc = multiclass_auc(test_labels, gb_multi_probs)
print(f'Gradient boosting OvR (3-class) → AUROC: {gb_multi_auc:.3f}, macro-F1: {gb_multi_f1:.3f}')
